# JET.jl Workshop

Shuhei Kadowaki

<h2>Workshop Outline<span class="tocSkip"></span></h2>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview-of-JET.jl" data-toc-modified-id="Overview-of-JET.jl-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview of JET.jl</a></span><ul class="toc-item"><li><span><a href="#What-is-JET-?" data-toc-modified-id="What-is-JET-?-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>What is JET ?</a></span></li><li><span><a href="#Why-JET-?" data-toc-modified-id="Why-JET-?-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Why JET ?</a></span></li></ul></li><li><span><a href="#Get-Started-!" data-toc-modified-id="Get-Started-!-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get Started !</a></span><ul class="toc-item"><li><span><a href="#Type-Level-Analysis" data-toc-modified-id="Type-Level-Analysis-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Type-Level Analysis</a></span></li><li><span><a href="#First-Class-Metaprogramming-Support" data-toc-modified-id="First-Class-Metaprogramming-Support-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>First-Class Metaprogramming Support</a></span></li><li><span><a href="#Entry-Points" data-toc-modified-id="Entry-Points-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Entry Points</a></span></li></ul></li><li><span><a href="#JET-on-Real-world-Package" data-toc-modified-id="JET-on-Real-world-Package-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>JET on Real-world Package</a></span></li><li><span><a href="#Performance-Linting" data-toc-modified-id="Performance-Linting-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Performance Linting</a></span></li><li><span><a href="#Performance-Linting-Example:-Improve-The-Performance-of-Julia-Compiler" data-toc-modified-id="Performance-Linting-Example:-Improve-The-Performance-of-Julia-Compiler-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Performance Linting Example: Improve The Performance of Julia Compiler</a></span></li></ul></div>

## Overview of JET.jl

### What is JET ?

- JET is a code analyzer for Julia
- JET can take an ordinal Julia program and _automatically_ detect possible error points – there is no need for any additional setups !

### Why JET ?

- JET's analysis is powered by type inference, so JET can detect _type-level_ errors (but without any additional annotations)
- JET offers a first-class metaprogramming support

## Get Started !

In [1]:
using JET

In [2]:
# NOTE:
# This is a very specific setup for this workshop.
# We usually just use the entries like `report_file`, `report_package` or `@report_call`, that are exported by JET default

# a sandbox module where JET's virtual process runs in
module __JET_VMOD__ end

# takes arbitrary Julia expression and runs JET on it
macro report_toplevel(xs...)
    iskwarg(@nospecialize(x)) = Meta.isexpr(x, :(=))

    jetconfigs = filter(iskwarg, xs)

    xs′ = filter(!iskwarg, xs)
    @assert length(xs′) == 1
    ex = first(xs′)
    toplevelex = (Meta.isexpr(ex, :block) ?
                  Expr(:toplevel, __source__, ex.args...) : # flatten here
                  Expr(:toplevel, __source__, ex)
                  ) |> QuoteNode

    return :(let
        analyzer = $(GlobalRef(JET, :JETAnalyzer))(; $(map(esc, jetconfigs)...))
        config = $(GlobalRef(JET, :ToplevelConfig))(; $(map(esc, jetconfigs)...))
        res = $(GlobalRef(JET, :virtual_process))($toplevelex,
                                                  $(string(__source__.file)),
                                                  analyzer,
                                                  config,
                                                  )
        $(GlobalRef(JET, :report_result))(stdout::IO, res; $(map(esc, jetconfigs)...))
    end)
end

@report_toplevel (macro with 1 method)

### Type-Level Analysis

As the first example, let's run JET on a simple function that calculates nth number of the Fibonacci series.

In [3]:
@report_toplevel begin
    fib(n) = n ≤ 2 ? n : fib(n-1) + fib(n-2)

    fib(1000)
    fib(1000.0)
    fib(BigInt(1000))
    fib("1000")       # error !
end

═════ 1 possible error found ═════
┌ @ In[3]:7 fib("1000")
│┌ @ In[3]:2 ≤(n, 2)
││┌ @ operators.jl:401 Base.<(x, y)
│││┌ @ operators.jl:352 Base.isless(x, y)
││││ no matching method found for call signature (Tuple{typeof(isless), String, Int64}): Base.isless(x::String, y::Int64)
│││└────────────────────


(included_files = Set{String}(), nreported = 1)

- JET successfully found an error on `fib("1000")`
- But `fib` is very inefficient, so `fib(1000)` won't terminate in actual execution

How is that possible ?

- JET analyzes an input program only on _type-level_ (so this is a "static analysis" using the technique called "abstract interpretation")
- JET's analysis is really different from actually running the program

To see the difference better, let's consider the next example.

In [4]:
# takes an iterator, and tries to parse its elements into floating numbers,
# and then sum up succesfully parsed elements
function parsesum(args)
    argvals = []
    for arg in args
        try
            push!(argvals, parse(Float64, arg))
        catch
        end
    end
    return sum(argvals)
end

parsesum(["20", "foo", "22."])

42.0

It works ! Let's see what JET says on it:

In [5]:
@report_toplevel begin
    function parsesum(args)
        argvals = []
        for arg in args
            try
                push!(argvals, parse(Float64, arg))
            catch
            end
        end
        return sum(argvals)
    end
    
    parsesum(["20", "foo", "22."])
end

═════ 1 possible error found ═════
┌ @ In[5]:13 parsesum(Base.vect("20", "foo", "22."))
│┌ @ In[5]:10 sum(argvals)
││┌ @ reducedim.jl:889 Base.#sum#738(Base.:, Base.pairs(Core.NamedTuple()), #self#, a)
│││┌ @ reducedim.jl:889 Base._sum(a, dims)
││││┌ @ reducedim.jl:893 Base.#_sum#740(Base.pairs(Core.NamedTuple()), #self#, a, _3)
│││││┌ @ reducedim.jl:893 Base._sum(Base.identity, a, Base.:)
││││││┌ @ reducedim.jl:894 Base.#_sum#741(Base.pairs(Core.NamedTuple()), #self#, f, a, _4)
│││││││┌ @ reducedim.jl:894 Base.mapreduce(f, Base.add_sum, a)
││││││││┌ @ reducedim.jl:322 Base.#mapreduce#731(Base.:, Base._InitialValue(), #self#, f, op, A)
│││││││││┌ @ reducedim.jl:322 Base._mapreduce_dim(f, op, init, A, dims)
││││││││││┌ @ reducedim.jl:330 Base._mapreduce(f, op, Base.IndexStyle(A), A)
│││││││││││┌ @ reduce.jl:402 Base.mapreduce_empty_iter(f, op, A, Base.IteratorEltype(A))
││││││││││││┌ @ reduce.jl:353 Base.reduce_empty_iter(Base.MappingRF(f, op), itr, ItrEltype)
│││││││││││││┌ @ reduce.jl

(included_files = Set{String}(), nreported = 1)

Why does JET complain something here ?

=> `argvals` is an untyped vector (i.e. `Vector{Any}`), so Julia doesn't know what `sum(argvals)` should return when it's empty.

In [6]:
# JET found this error ahead of time !
parsesum(["twenty", "foo", "twenty-two"])

LoadError: MethodError: no method matching zero(::Type{Any})
[0mClosest candidates are:
[0m  zero(::Type{Union{Missing, T}}) where T at missing.jl:105
[0m  zero([91m::Union{Type{P}, P}[39m) where P<:Dates.Period at /Users/aviatesk/julia/julia/usr/share/julia/stdlib/v1.8/Dates/src/periods.jl:53
[0m  zero([91m::LinearAlgebra.UniformScaling{T}[39m) where T at /Users/aviatesk/julia/julia/usr/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:136
[0m  ...

- JET reasons about _all the possible program executions_
- so it can detect possible errors that test cases may not cover yet

In [7]:
@report_toplevel begin
    function parsesum(args)
        argvals = Float64[] # tell the compiler about the element types
        for arg in args
            try
                push!(argvals, parse(Float64, arg))
            catch
            end
        end
        return sum(argvals)
    end
    
    parsesum(["twenty", "foo", "twenty-two"])
end

No errors !


(included_files = Set{String}(), nreported = 0)

This feature of JET is particularly useful when you're defining a new type that implements some interfaces so that we can use it for existing generic functions.

In [8]:
mutable struct Squares
    count::Int
end
Base.iterate(S::Squares, s=1) =
    s > S.count ? nothing : (s^2, s+1)
Base.length(S::Squares) = S.count

sum(Squares(10))

385

`Squares` behaves like an iterator that yields squared numbers.
It seemingly works well for the `sum` function, but JET should tell us that it has the same problem as `parsesum`.

In [9]:
@report_toplevel context=__JET_VMOD__ virtualize=false begin
    mutable struct Squares
        count::Int
    end
    Base.iterate(S::Squares, s=1) =
        s > S.count ? nothing : (s^2, s+1)
    Base.length(S::Squares) = S.count

    sum(Squares(10))
end

═════ 1 possible error found ═════
┌ @ In[9]:9 Main.__JET_VMOD__.sum(Main.__JET_VMOD__.Squares(10))
│┌ @ reduce.jl:532 Base.#sum#256(Base.pairs(Core.NamedTuple()), #self#, a)
││┌ @ reduce.jl:532 Base.sum(Base.identity, a)
│││┌ @ reduce.jl:503 Base.#sum#255(Base.pairs(Core.NamedTuple()), #self#, f, a)
││││┌ @ reduce.jl:503 Base.mapreduce(f, Base.add_sum, a)
│││││┌ @ reduce.jl:289 Base.#mapreduce#252(Base.pairs(Core.NamedTuple()), #self#, f, op, itr)
││││││┌ @ reduce.jl:289 Base.mapfoldl(f, op, itr)
│││││││┌ @ reduce.jl:162 Base.#mapfoldl#248(Base._InitialValue(), #self#, f, op, itr)
││││││││┌ @ reduce.jl:162 Base.mapfoldl_impl(f, op, init, itr)
│││││││││┌ @ reduce.jl:44 Base.foldl_impl(op′, nt, itr′)
││││││││││┌ @ reduce.jl:49 Base.reduce_empty_iter(op, itr)
│││││││││││┌ @ reduce.jl:356 Base.reduce_empty_iter(op, itr, Base.IteratorEltype(itr))
││││││││││││┌ @ reduce.jl:357 Base.reduce_empty(op, Base.eltype(itr))
│││││││││││││┌ @ reduce.jl:330 Base.reduce_empty(Base.getproperty(op, :rf),

(included_files = Set{String}(), nreported = 1)

This is essentially because `Squared` actually missed to implement one more required interface;
we can implement the `eltype` interface so that Julia can understand the return value for `sum(Squares(0))`.

In [10]:
@report_toplevel context=__JET_VMOD__ virtualize=false begin
    Base.eltype(::Squares) = Int

    sum(Squares(10))
end

No errors !


(included_files = Set{String}(), nreported = 0)

- Julia doesn't guarantee the functionality of an input program — Julia programmers have to make sure it works by themselves
- But test may not cover all the possible cases
- JET can check the quality of generic code in a way that doesn't rely on runtime !

### First-Class Metaprogramming Support

Julia offers various forms of metaprogramming.

Currently we have:
- `eval` and macros for abstract syntax level code generation
- `@generated` function for code generation at inference stage

- `eval` is great for eliminating syntax-level duplications
- Macros are great for providing simple DSLs and/or optimizations

For example, let's see the code snippet below adapted from [Julia base](https://github.com/JuliaLang/julia/blob/9cf1c3a795e5ca40101ffd3670511041145a415c/base/reducedim.jl#L885-L896):
```julia
for (fname, _fname, op) in [(:sum,     :_sum,     :add_sum),
                            (:prod,    :_prod,    :mul_prod),
                            (:maximum, :_maximum, :max),
                            (:minimum, :_minimum, :min)]
    @eval begin
        @inline ($fname)(a::AbstractArray; dims=:, kw...) =
            ($_fname)(a, dims; kw...)
        @inline ($fname)(f, a::AbstractArray; dims=:, kw...) = 
            ($_fname)(f, a, dims; kw...)

        ($_fname)(a, ::Colon; kw...) = ($_fname)(identity, a, :; kw...)
        ($_fname)(f, a, ::Colon; kw...) = mapreduce(f, $(GlobalRef(Base, op)), a; kw...)
    end
end
```

But static analyzers in general suffers from this kind of code:
- it has to actually _execute_ a program construction that involves `eval`
- it needs to reason on programs after macros are expanded

JET deals with this using the technique called "partial evaluation".
That is, JET selectively executes parts of input program that include "definitions".
But at the same time, it won't execute the "usages" of those definitions, but analyze them statically instead.

Since JET's static analysis uses "abstract interpretation", I use "concretization" when referring to JET's partial evaluations of "definitions".

In [11]:
@report_toplevel begin
    # JET actually evaluates this top-level code block ("definitions")
    for (fname, _fname, op) in [(:sum,     :_sum,     :add_sum),
                                (:prod,    :_prod,    :mul_prod),
                                (:maximum, :_maximum, :max),
                                (:minimum, :_minimum, :min)]
        fname, _fname = Symbol.(string.("my", (fname, _fname)))
        @eval begin
            @inline ($fname)(a::AbstractArray; dims=:, kw...) =
                ($_fname)(a, dims; kw...)
            @inline ($fname)(f, a::AbstractArray; dims=:, kw...) = 
                ($_fname)(f, a, dims; kw...)

            ($_fname)(a, ::Colon; kw...) = ($_fname)(identity, a, :; kw...)
            ($_fname)(f, a, ::Colon; kw...) = mapreduce(f, $(GlobalRef(Base, op)), a; kw...)
        end
    end

    # JET analyzes these top-level calls ("usages")
    mysum([0,1,2,3])
    myprod(rand(1000000000))
    ary = rand(1000000000)
    mymaximum(ary)
    myminimum(ary)
end

No errors !


(included_files = Set{String}(), nreported = 0)

Similarly, macros are fully supported. JET understands macro definitions, and expand all the macro calls appearing within input program.

In [12]:
@report_toplevel begin
    macro defv(v)
        return :(global v = $v)
    end

    @defv 1000
    println(v)
end

No errors !


(included_files = Set{String}(), nreported = 0)

Caveat: so JET isn't "safe" – it's not fully static and its analysis involves actual program execution of user code.
So for example, if a macro expansion never terminates, JET's analysis also never terminates.

```julia
# this analysis won't terminate until 10000 seconds elapsed ...
@report_toplevel begin
    macro defv(v)
        sleep(10000)
        return :(global v = $v)
    end

    @defv 1000
    println(v)
end
```

If an error happened during concretization, JET simply reports the error that actually happened.

In [13]:
@report_toplevel begin
    macro badshow(val)
        :(@show $value) # typo: val
    end
    
    @badshow sin(10)
end

═════ 1 toplevel error found ═════
┌ @ In[13]:6 
│ UndefVarError: value not defined
│ Stacktrace:
│  [1] var"@badshow"(__source__::LineNumberNode, __module__::Module, val::Any)
│    @ Main ./In[13]:3
└────────────


(included_files = Set{String}(), nreported = 1)

- `@generated` allows us to intervene into inference, do some code generation using type information
- it's used for advanced optimizations, or customized code execution (e.g. autodiff)

An example usage adapted from [Julia base](https://github.com/JuliaLang/julia/blob/018977209bb4fd707ec61c59dfd31860abaa6717/base/namedtuple.jl#L253):
```julia
@generated function merge(a::NamedTuple{an}, b::NamedTuple{bn}) where {an, bn}
    names = Base.merge_names(an, bn)
    types = Base.merge_types(names, a, b)
    vals = Any[ 
    :(getfield($(Base.sym_in(names[n], bn) ? :b : :a),
               $(QuoteNode(names[n])))) 
    for n in 1:length(names)]
        return :( NamedTuple{$names,$types}(($(vals...),)) )
    end
end
```

But staged programming (rather, metaprogramming in general) is very tricky.
For instance, if we've forgot to add `QuoteNode` within the `merge` function, then it simply doesn't work:

In [14]:
@generated function merge(a::NamedTuple{an}, b::NamedTuple{bn}) where {an, bn}
    names = Base.merge_names(an, bn)
    types = Base.merge_types(names, a, b)
    vals = Any[ 
        :(getfield($(Base.sym_in(names[n], bn) ? :b : :a),
                   $(names[n]))) # we forgot to quote these names !
        for n in 1:length(names)]
    return :( NamedTuple{$names,$types}(($(vals...),)) )
end

merge((; a = 10), (; a = 2, b = 10))

LoadError: TypeError: in getfield, expected Symbol, got a value of type NamedTuple{(:a,), Tuple{Int64}}

JET reuses Julia's native type inference, so it can also perfectly reason about `@generated` functions and detect possible errors within them !

In [15]:
@report_toplevel begin
    @generated function merge(a::NamedTuple{an}, b::NamedTuple{bn}) where {an, bn}
        names = Base.merge_names(an, bn)
        types = Base.merge_types(names, a, b)
        vals = Any[ 
            :(getfield($(Base.sym_in(names[n], bn) ? :b : :a),
                       $(names[n])))
            for n in 1:length(names)]
        return :( NamedTuple{$names,$types}(($(vals...),)) )
    end
    
    merge((; a = 10), (; a = 2, b = 10))
end

═════ 1 possible error found ═════
┌ @ In[15]:12 merge(Core.apply_type(Core.NamedTuple, Core.tuple(:a))(Core.tuple(10)), Core.apply_type(Core.NamedTuple, Core.tuple(:a, :b))(Core.tuple(2, 10)))
│┌ @ In[15]:0 getfield(b, a)
││ invalid builtin function call: getfield(b::NamedTuple{(:a, :b), Tuple{Int64, Int64}}, a::NamedTuple{(:a,), Tuple{Int64}})
│└────────────


(included_files = Set{String}(), nreported = 1)

- Metaprogramming is one of the most important features of Julia, and it brings Julia's extreme expressivity
- Metaprogramming is tricky and can be a nest of bugs – but static analyzer in general suffers from it
- JET's partial concretization allows us to analyze Julia programs effectively no matter how heavily they use metaprogramming features !

### Entry Points

- `report_file("path/to/file")`: analyzes a script (it's supposed to include both "definitions" and "usages")
- `report_package("package_name")`: analyzers a package (it's okay if it only includes "definitions")
- `report_call(f, argtypes)` / `@report_call f(args...)`: analyzes a generic function call (similar to `code_typed` and its family, and very handy for interactive analysis)

In [16]:
@report_call sum("julia") # equivalent to report_call(sum, (String,))

═════ 2 possible errors found ═════
┌ @ reduce.jl:532 Base.#sum#256(Base.pairs(Core.NamedTuple()), #self#, a)
│┌ @ reduce.jl:532 Base.sum(Base.identity, a)
││┌ @ reduce.jl:503 Base.#sum#255(Base.pairs(Core.NamedTuple()), #self#, f, a)
│││┌ @ reduce.jl:503 Base.mapreduce(f, Base.add_sum, a)
││││┌ @ reduce.jl:289 Base.#mapreduce#252(Base.pairs(Core.NamedTuple()), #self#, f, op, itr)
│││││┌ @ reduce.jl:289 Base.mapfoldl(f, op, itr)
││││││┌ @ reduce.jl:162 Base.#mapfoldl#248(Base._InitialValue(), #self#, f, op, itr)
│││││││┌ @ reduce.jl:162 Base.mapfoldl_impl(f, op, init, itr)
││││││││┌ @ reduce.jl:44 Base.foldl_impl(op′, nt, itr′)
│││││││││┌ @ reduce.jl:48 Base._foldl_impl(op, nt, itr)
││││││││││┌ @ reduce.jl:62 op(v, Base.getindex(y, 1))
│││││││││││┌ @ reduce.jl:81 Base.getproperty(op, :rf)(acc, x)
││││││││││││┌ @ reduce.jl:24 Base.+(x, y)
│││││││││││││ no matching method found for call signature (Tuple{typeof(+), Char, Char}): Base.+(x::Char, y::Char)
││││││││││││└────────────────
│││││

(Char, 2)

## JET on Real-world Package

Now let's run JET on a package in the wild.

In this workshop I will take [MacroTools.jl](https://github.com/FluxML/MacroTools.jl) as the target.
This is a very useful package for developing complex macros or general manipulations of Julia AST.
And it's also very important for the ecosystem – there are more than [1000 direct/indirect dependents](https://juliahub.com/ui/Packages/MacroTools/38lnR/0.5.6?t=2) of the package !

We will use the `report_package` entry to analyze a package:
- it tries to find a package in the active environment, and then analyzes its code base
- after collecting "definitions", JET enters analysis from the "stub usages" using the signatures of methods (so there is no need for test cases)

```julia
g(a) = compute(a) # => analyze g(::Any)
f() = g(10)       # => analyze f() (, which includes analysis of g(::Int))
```

In [17]:
using Pkg

let back = pwd()
    try 
        Pkg.develop("MacroTools")
        path = Base.find_package("MacroTools")
        cd(dirname(dirname(path)))
        run(`git checkout -b juliacon fef1c6fab11f30d6ae1c1fff6c42be49ccef5d10`)
    catch err
        @warn "please manually checkout to fef1c6fab11f30d6ae1c1fff6c42be49ccef5d10"
        throw(err)
    finally
        cd(back)
    end
end

   Resolving package versions...
    Updating `~/julia/juliacon2021/workshop/Project.toml`
  [1914dd2f] ~ MacroTools v0.5.6 ⇒ v0.5.6 `~/julia/packages/MacroTools`
    Updating `~/julia/juliacon2021/workshop/Manifest.toml`
  [1914dd2f] ~ MacroTools v0.5.6 ⇒ v0.5.6 `~/julia/packages/MacroTools`
  [0dad84c5] ~ ArgTools ⇒ v1.1.1
  [f43a241f] ~ Downloads ⇒ v1.4.0
  [b27032c2] ~ LibCURL ⇒ v0.6.2
  [ca575930] ~ NetworkOptions ⇒ v1.2.0
  [44cfe95a] ~ Pkg ⇒ v1.6.0
  [fa267f1f] ~ TOML ⇒ v1.0.0
  [a4e569a6] ~ Tar ⇒ v1.9.1
  [e66e0078] ~ CompilerSupportLibraries_jll ⇒ v0.4.0+1
  [deac9b47] ~ LibCURL_jll ⇒ v7.73.0+4
  [29816b5a] ~ LibSSH2_jll ⇒ v1.9.1+2
  [c8ffd9c3] ~ MbedTLS_jll ⇒ v2.24.0+2
  [14a3606d] ~ MozillaCACerts_jll ⇒ v2020.7.22
  [4536629a] - OpenBLAS_jll
  [83775a58] ~ Zlib_jll ⇒ v1.2.12+1
  [8e850b90] ~ libblastrampoline_jll ⇒ v3.0.4+0
  [8e850ede] ~ nghttp2_jll ⇒ v1.41.0+1
  [3f19e933] ~ p7zip_jll ⇒ v16.2.1+1
Switched to a new branch 'juliacon'


Process(`git checkout -b juliacon fef1c6fab11f30d6ae1c1fff6c42be49ccef5d10`, ProcessExited(0))

In [18]:
report_package("MacroTools")

[toplevel-info] applied JET configurations in /Users/aviatesk/julia/packages/.JET.toml
[toplevel-info] virtualized the context of Main (took 0.007 sec)
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/MacroTools.jl
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/match/match.jl
[toplevel-info]  exited from /Users/aviatesk/julia/packages/MacroTools/src/match/match.jl (took 0.04 sec)
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/match/types.jl
[toplevel-info]  exited from /Users/aviatesk/julia/packages/MacroTools/src/match/types.jl (took 0.014 sec)
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/match/union.jl
[toplevel-info]  exited from /Users/aviatesk/julia/packages/MacroTools/src/match/union.jl (took 0.007 sec)
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/match/macro.jl
[toplevel-info]  exited from /Users/aviatesk/julia/packages/MacroTools/src/matc

[toplevel-info] analyzing from top-level definitions ... 156/156
═════ 11 possible errors found ═════
┌ @ /Users/aviatesk/julia/packages/MacroTools/src/match/match.jl:27 Base.getproperty(Base.getproperty(MacroTools.Base, :match::Symbol)::typeof(match)(r"^@?(.*?)_+(_str)?$", MacroTools.string(s::Symbol)::String)::Union{Nothing, RegexMatch}, :captures::Symbol)
│┌ @ Base.jl:42 Base.getfield(x::Nothing, f::Symbol)
││ type Nothing has no field captures
│└──────────────
┌ @ /Users/aviatesk/julia/packages/MacroTools/src/match/types.jl:20 MacroTools.map(MacroTools.totype, ts::Vector{Symbol})
│┌ @ abstractarray.jl:2853 Base.collect_similar(A::Vector{Symbol}, Base.Generator(f::typeof(MacroTools.totype), A::Vector{Symbol})::Base.Generator{Vector{Symbol}, typeof(MacroTools.totype)})
││┌ @ array.jl:698 Base._collect(cont::Vector{Symbol}, itr::Base.Generator{Vector{Symbol}, typeof(MacroTools.totype)}, Base.IteratorEltype(itr::Base.Generator{Vector{Symbol}, typeof(MacroTools.totype)})::Base.EltypeUnk

(included_files = Set(["/Users/aviatesk/julia/packages/MacroTools/src/examples/threading.jl", "/Users/aviatesk/julia/packages/MacroTools/src/examples/forward.jl", "/Users/aviatesk/julia/packages/MacroTools/src/MacroTools.jl", "/Users/aviatesk/julia/packages/MacroTools/src/match/match.jl", "/Users/aviatesk/julia/packages/MacroTools/src/utils.jl", "/Users/aviatesk/julia/packages/MacroTools/src/match/macro.jl", "/Users/aviatesk/julia/packages/MacroTools/src/structdef.jl", "/Users/aviatesk/julia/packages/MacroTools/src/match/union.jl", "/Users/aviatesk/julia/packages/MacroTools/src/examples/destruct.jl", "/Users/aviatesk/julia/packages/MacroTools/src/match/types.jl"]), nreported = 11)

There are bunch of errors ! Let's fix them.

_Live coding (5 mins)_

In [19]:
report_package("MacroTools")

[toplevel-info] applied JET configurations in /Users/aviatesk/julia/packages/.JET.toml
[toplevel-info] virtualized the context of Main (took 0.002 sec)
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/MacroTools.jl
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/match/match.jl
[toplevel-info]  exited from /Users/aviatesk/julia/packages/MacroTools/src/match/match.jl (took 0.06 sec)
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/match/types.jl
[toplevel-info]  exited from /Users/aviatesk/julia/packages/MacroTools/src/match/types.jl (took 0.026 sec)
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/match/union.jl
[toplevel-info]  exited from /Users/aviatesk/julia/packages/MacroTools/src/match/union.jl (took 0.01 sec)
[toplevel-info] entered into /Users/aviatesk/julia/packages/MacroTools/src/match/macro.jl
[toplevel-info]  exited from /Users/aviatesk/julia/packages/MacroTools/src/match

[toplevel-info] analyzing from top-level definitions ... 156/156
═════ 2 possible errors found ═════
┌ @ /Users/aviatesk/julia/packages/MacroTools/src/utils.jl:21 Base.collect(Base.Generator(#13::MacroTools.var"#13#14", MacroTools.map(MacroTools.esc, xs::Tuple)::Any)::Base.Generator{_A, MacroTools.var"#13#14"} where _A)
│┌ @ array.jl:775 Base.collect_to_with_first!(Base._array_for(Base.typeof(v1::Expr)::Type{Expr}, Base.getproperty(itr::Base.Generator{_A, MacroTools.var"#13#14"} where _A, :iter::Symbol)::Any, isz::Any)::Any, v1::Expr, itr::Base.Generator{_A, MacroTools.var"#13#14"} where _A, st::Any)
││┌ @ array.jl:799 Base.grow_to!(dest::Any, itr::Base.Generator{_A, MacroTools.var"#13#14"} where _A, st::Any)
│││┌ @ dict.jl:153 Base.indexed_iterate(Core.getfield(Base.indexed_iterate(y::Tuple{Expr, Any}, 1)::Tuple{Expr, Int64}, 1)::Expr, 1)
││││┌ @ tuple.jl:91 Base.iterate(I::Expr)
│││││ no matching method found for call signature (Tuple{typeof(iterate), Expr}): Base.iterate(I::Expr)
││

(included_files = Set(["/Users/aviatesk/julia/packages/MacroTools/src/examples/threading.jl", "/Users/aviatesk/julia/packages/MacroTools/src/examples/forward.jl", "/Users/aviatesk/julia/packages/MacroTools/src/MacroTools.jl", "/Users/aviatesk/julia/packages/MacroTools/src/match/match.jl", "/Users/aviatesk/julia/packages/MacroTools/src/utils.jl", "/Users/aviatesk/julia/packages/MacroTools/src/match/macro.jl", "/Users/aviatesk/julia/packages/MacroTools/src/structdef.jl", "/Users/aviatesk/julia/packages/MacroTools/src/match/union.jl", "/Users/aviatesk/julia/packages/MacroTools/src/examples/destruct.jl", "/Users/aviatesk/julia/packages/MacroTools/src/match/types.jl"]), nreported = 2)

Yay, now we've greatly improved the quality of MacroTools.jl by fixing the actual bugs and improving type stabilities !

## Performance Linting

- JET offers an infrastructure for "plugin" analyzers
- JET's default error analyzer (, which we've used so far) is actually one specific instance of them
- Another example: [JETTest.jl's Performance Linter](https://github.com/aviatesk/JETTest.jl)

- JET's error analyzer aims at finding possible "error"s, by analyzing type inference
- The performance linter aims at finding possible performance pitfalls (currently it mostly accounts for runtime dispatches), by analyzing optimizations

In [20]:
# calculates cumulative sum starting from 1 to n
function f(n::Integer)
    ary = [] # ::Vector{Any}
    push!(ary, 1)
    for i in 2:n
        push!(ary, i+ary[i-1]) # valid call, but not performative
    end
    return ary
end

f(10)
@time f(10000000);

  0.915732 seconds (20.00 M allocations: 451.999 MiB, 41.18% gc time)


In [21]:
@report_call f(10000000)

No errors !


(Vector{Any}, 0)

In [22]:
using JETTest

@report_dispatch f(10000000)

═════ 1 possible error found ═════
┌ @ In[20]:6 Main.+(%31, %34)
│ runtime dispatch detected: Main.+(%31::Int64, %34::Any)
└────────────


(Vector{Any}, 1)

In [23]:
function f2(n::T) where T<:Integer
    ary = T[] # very type stable !
    push!(ary, 1)
    for i in 2:n
        push!(ary, i+ary[i-1]) # performative !
    end
    return ary
end

f2 (generic function with 1 method)

In [24]:
@report_dispatch f2(10000000)

No errors !


(Vector{Int64}, 0)

In [25]:
f2(10)
@time f2(10000000);

  0.224100 seconds (18 allocations: 146.831 MiB, 4.46% gc time)


The performance linter is more powerful than a manual inspection with `code_typed` or `code_warntype`; it can analyze the entire call graph rather than just look at the final output of the compilation pipeline.

In [26]:
callf(n) = f(n)

callf (generic function with 1 method)

In [27]:
@code_warntype callf(10000000)

MethodInstance for callf(::Int64)
  from callf(n) in Main at In[26]:1
Arguments
  #self#::Core.Const(callf)
  n::Int64
Body::Vector{Any}
1 ─ %1 = Main.f(n)::Vector{Any}
└──      return %1



In [28]:
@report_dispatch callf(10000000)

═════ 1 possible error found ═════
┌ @ In[26]:1 Main.f(n)
│┌ @ In[20]:6 Main.+(%31, %34)
││ runtime dispatch detected: Main.+(%31::Int64, %34::Any)
│└────────────


(Vector{Any}, 1)

## Performance Linting Example: Improve The Performance of Julia Compiler

Let's run the performance linter on Julia compiler itself – we all want to JIT to be faster !

In [1]:
using JETTest

const CC = Core.Compiler

# some functions in the compiler uses dispatch to branch on input types
# they can be rewritten so that they won't be dynamically dispatched,
# but we just ignore them for this workshop
function function_filter(@nospecialize(ft))
    ft === typeof(CC.widenconst) && return false
    ft === typeof(CC.widenconditional) && return false
    ft === typeof(CC.widenwrappedconditional) && return false
    ft === typeof(CC.maybe_extract_const_bool) && return false
    ft === typeof(CC.ignorelimited) && return false
    return true
end

# the compiler code uses "dummy" versions of some Base functions
# we just focus on the compiler code for this workshop
function frame_filter((; linfo) = sv)
    meth = linfo.def
    isa(meth, Method) || return true
    return occursin("compiler/", string(meth.file))
end

# typeinf(::NativeInterpreter, ::InferenceState) is an entry of type inference
# we analyze dispatches that are involved within a static call graph of it
report_dispatch(CC.typeinf, (CC.NativeInterpreter, CC.InferenceState); function_filter, frame_filter)

# # we will also use Revise to interactively reflect the changes made on source code to JET's analysis
# using Revise
# Revise.track(Core.Compiler)

═════ 148 possible errors found ═════
┌ @ compiler/typeinfer.jl:205 Core.Compiler._typeinf(interp, frame)
│┌ @ compiler/typeinfer.jl:226 Core.Compiler.typeinf_nocycle(interp, frame)
││┌ @ compiler/abstractinterpretation.jl:1968 Core.Compiler.typeinf_local(interp, frame)
│││┌ @ compiler/abstractinterpretation.jl:1758 Core.Compiler.abstract_eval_value(interp, condx, changes, frame)
││││┌ @ compiler/abstractinterpretation.jl:1451 Core.Compiler.abstract_eval_special_value(interp, e, vtypes, sv)
│││││┌ @ compiler/abstractinterpretation.jl:1437 Core.Compiler.abstract_eval_ssavalue(Core.typeassert(e, Core.Compiler.SSAValue), Core.Compiler.getproperty(sv, :src))
││││││┌ @ compiler/abstractinterpretation.jl:1640 Core.Compiler.getindex(%1, %2)
│││││││ runtime dispatch detected: Core.Compiler.getindex(%1::Any, %2::Int64)
││││││└───────────────────────────────────────────
│││┌ @ compiler/abstractinterpretation.jl:1786 Core.Compiler.conditional_changes(changes_else, Core.Compiler.getproperty(condt,

││││┌ @ compiler/ssair/passes.jl:850 Core.Compiler.isbitstype(%1465)
│││││ runtime dispatch detected: Core.Compiler.isbitstype(%1465::Any)
││││└────────────────────────────────
│││┌ @ compiler/optimize.jl:309 Core.Compiler.type_lift_pass!(ir)
││││┌ @ compiler/ssair/passes.jl:1062 Core.Compiler.convert(Core.Compiler.Int, %320)
│││││ runtime dispatch detected: Core.Compiler.convert(Core.Compiler.Int, %320::Any)
││││└─────────────────────────────────
││││┌ @ compiler/ssair/passes.jl:1069 Core.Compiler.push!(%114, %362)
│││││ runtime dispatch detected: Core.Compiler.push!(%114::Vector{Tuple{Int64, Int64, Core.SSAValue, Int64}}, %362::Tuple{Int64, Int64, Union{Bool, Core.SSAValue}, Int64})
││││└─────────────────────────────────
│││┌ @ compiler/optimize.jl:313 Core.Compiler.verify_ir(ir)
││││┌ @ compiler/ssair/verify.jl:67 #self#(ir, true)
│││││┌ @ array.jl:413 Core.Compiler.setindex!(%414, %412, 1)
││││││ runtime dispatch detected: Core.Compiler.setindex!(%414::Vector{Int64}, %412::Any, 1)


(Bool, 148)

So many runtime dispatches. Fix them as long as time permits.

_Live coding starts_